# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234069383028                   -0.50    8.0
  2   -7.250219037262       -1.79       -1.40    1.0
  3   -7.251290879901       -2.97       -2.10    4.0
  4   -7.251225461381   +   -4.18       -2.12    4.0
  5   -7.251326377243       -4.00       -2.62    3.0
  6   -7.251337753939       -4.94       -3.11    2.0
  7   -7.251338760807       -6.00       -3.95    2.0
  8   -7.251338790347       -7.53       -4.07    4.0
  9   -7.251338796836       -8.19       -4.43    1.0
 10   -7.251338798390       -8.81       -4.79    2.0
 11   -7.251338798664       -9.56       -5.28    3.0
 12   -7.251338798704      -10.40       -6.04    3.0
 13   -7.251338798704      -12.35       -6.42    2.0
 14   -7.251338798705      -13.20       -7.08    3.0
 15   -7.251338798705      -14.75       -7.47    3.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.044110055482530934
[ Info: Arnoldi iteration step 2: normres = 0.37894118829383533
[ Info: Arnoldi iteration step 3: normres = 0.8692458631361217
[ Info: Arnoldi iteration step 4: normres = 0.4559716000434186
[ Info: Arnoldi iteration step 5: normres = 0.5409012023889492
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.19e-01, 8.30e-02, 4.93e-01, 1.70e-01, 6.02e-03)
[ Info: Arnoldi iteration step 6: normres = 0.2948106091347428
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (2.96e-02, 1.55e-01, 1.87e-01, 1.34e-01, 8.93e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06958833373148152
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (9.67e-04, 1.01e-02, 9.26e-03, 3.48e-02, 4.85e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11389176510541847
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.79e